# Not all movies have a foreign release!!~~~!!!!!!

# Fields I will probably want to get for predicting international success (Do I want to look at prediction pre-release? or after first weekend?
### Will probably want to be looking at predicting this pre-release or after opening weekend to have any real value to investors/studios

* Domestic Total Gross (maybe...but not really relevant to problem)
* Opening weekend (for predicting success after domestic opening weekend)
* Release Date
* Distributor
* Runtime
* Production Budget
* Genre
* MPAA Rating
* Director
* Writers
* Producers
* Actors (maybe have another analysis for actor success?)
* 3D?
* IMAX?
* Brand

## Target variable for training and prediction:

* Foreign total gross

# List of movies 2016:

(1-100) http://www.boxofficemojo.com/yearly/chart/?view=releasedate&view2=domestic&yr=2016&sort=gross&order=DESC&studio= 

(101 - 200) http://www.boxofficemojo.com/yearly/chart/?page=2&view=releasedate&view2=domestic&yr=2016&p=.htm

(201-300) http://www.boxofficemojo.com/yearly/chart/?page=3&view=releasedate&view2=domestic&yr=2016&p=.htm

(301-400) http://www.boxofficemojo.com/yearly/chart/?page=4&view=releasedate&view2=domestic&yr=2016&p=.htm

# List of movies 2015: 

(1-100) http://www.boxofficemojo.com/yearly/chart/yr=2015&view=releasedate&view2=domestic&sort=gross&order=DESC&&p=.htm

(101-200) http://www.boxofficemojo.com/yearly/chart/page=2&view=releasedate&view2=domestic&yr=2015&p=.htm

(201-300) http://www.boxofficemojo.com/yearly/chart/page=3&view=releasedate&view2=domestic&yr=2015&p=.htm

(301-400) http://www.boxofficemojo.com/yearly/chart/page=4&view=releasedate&view2=domestic&yr=2015&p=.htm

(401-500)
http://www.boxofficemojo.com/yearly/chart/page=5&view=releasedate&view2=domestic&yr=2015&p=.htm

# Example of clicking on a movie link in one of these lists

#### http://www.boxofficemojo.com/movies/?id=starwars7.htm

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests

%matplotlib inline

# Scraping the yearly movie table
* ToDo
    * Parse the input so the dollar values are stripped and turned to integers etc...
    * adjust scale to absolute single dollar amount
    * adjust ticket values
    * parse the % change column
    * need to find ways to deal with missing values

In [2]:
url_yearly = 'http://www.boxofficemojo.com/yearly/'

response_yearly = requests.get(url_yearly)

In [3]:
page_yearly = response_yearly.text
soup_yearly = BeautifulSoup(page_yearly)

/Users/williamcosby/anaconda/envs/data/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [4]:
soup_yearly.find('title')

<title>Movie Box Office Results by Year, 1980-Present - Box Office Mojo</title>

In [5]:
# movie_list = soup_yearly.find("table",{ "class" : "wikitable sortable plainrowheaders" })
movie_table = soup_yearly.find_all('table')
# print(movie_table)

In [6]:
movie_data = []

# header = ['Year','Total Gross','Change','Tickets Sold','Change','# of movies','Total Screens',
#           'Avg. Ticket Price','Avg. Cost','#1 Movie']
movie_data = []

# the first row is the title of the columns
for row in movie_table[2].findAll("tr"):
    movie_row = []
    for i,cell in enumerate(row.findAll("td")):
#         print cell
        movie_row.append(cell.find(text=True))
    movie_data.append(movie_row)

header = movie_data.pop(0)
movies_df = pd.DataFrame(movie_data,columns = header)
# movies_df.dropna()

In [7]:
movies_df.head()

,Year,Total,Change,Tickets,Change,# of,Total,Avg.,Avg.,#1 Movie
0,2016,"$6,169.2",-,719.0,-,362,-,$8.58,-,Finding Dory
1,2015,"$11,128.5",+7.4%,"1,320.1",+4.1%,701,-,$8.43,-,Star Wars: The Force Awakens
2,2014,"$10,360.8",-5.2%,"1,268.2",-5.6%,702,-,$8.17,-,American Sniper
3,2013,"$10,923.6",+0.8%,"1,343.6",-1.3%,688,-,$8.13,-,Catching Fire
4,2012,"$10,837.4",+6.5%,"1,361.5",+6.1%,667,-,$7.96,-,The Avengers


# Now looking at parsing individual movie pages

In [29]:
# if needed: pip install requests

url = 'http://www.boxofficemojo.com/movies/?id=marvel2016.htm'

response = requests.get(url)

In [30]:
response.status_code

200

In [31]:
page_lebowski = response.text
soup_lebowski = BeautifulSoup(page_lebowski)

# Movie Page Scraping function

In [15]:
import re
from collections import defaultdict()

# testing for 1 page first
def parse_movie_page(url_list):
    movie_dict = defaultdict()
    
    for url in url_list:
        
        response = requests.get(url)
        # should probably check to make sure the response code is good to continue
        
        #'''get the page and soup reference'''#
        movie_page = response.text
        movie_soup = BeautifulSoup(movie_page)
        
        
        # MOVIE NAME
        movie_name = movie_soup.find('td')[2].find('b').get_text(strip=True)

        #''' DOMESTIC TOTAL GROSS '''#
        # can do this the normal, easy way, also it is present for every movie
        dtg_string = movie_soup.find(text=re.compile('Domestic Total')) # finds the label
        # format the dtg
    #     dtg = dtg_string.findNextSibling().text # grabs the next value, which is the dtg
        dtg = dtg_string.find_parent("td").find('b').get_text(strip=True)
        dtg = dtg.replace('$','').replace(',','')
        movie_dict[movie_name].append(int(dtg))


        #''' DOMESTIC OPENING WEEKEND '''#
        domestic_opening_string = movie_soup.find_all(class_='mp_box_content')[1].find('tr').find_all('td')[1].get_text(strip=True)
        domestic_opening = domestic_opening_string.replace('$','').replace(',','')
        movie_dict[movie_name].append(int(domestic_opening))

        #''' FOREIGN GROSS TOTAL '''#
        # have to check if it has a foreign release
        # if it doesnt have a foreign release then just put in 0
        try:
            ftg_string = movie_soup.find(text="Foreign:").find_parent("td").find_next_sibling("td").get_text(strip=True)
            ftg = ftg_string.replace('$','').replace(',','')
            movie_dict[movie_name].append(int(ftg))
        except: # no foreign release data
            movie_dict[movie_name].append(0)


        #''' BUDGET '''#
        try:
            budget_string = movie_soup.find(text=re.compile('Production Budget:')).findNextSibling().get_text(strip=True)
            budget = budget_string.replace('$','').replace(',','')
            # split the string, use the 2nd index to determine scale (itll be like ['245','million'])
            if budget.split()[1] == 'million':
                movie_dict[movie_name].append(budget.split()[0]*1000000)
            elif budge.split()[1] == 'thousand':
                movie_dict[movie_name].append(budget.split()[0]*1000)
            else:
                print('THE BUDGET WAS ',budget.split())
        except:
            print("budget failed for ",movie_name," with string: ",movie_soup.find(text=re.compile('Production Budget:')))
            print
            movie_dict[movie_name].append(0)

        #''' GENRE '''#
        genre = movie_soup.find(text=re.compile('Genre:')).find_parent('td').find('b').get_text(strip=True)
        movie_dict[movie_name].append(genre)

    return movie_dict
    

In [28]:
# from collections import defaultdict
# movies = [('http://www.boxofficemojo.com/movies/?id=pixar2015.htm','Finding Dory'),
#           ('http://www.boxofficemojo.com/movies/?id=marvel2016.htm','Captain America: Civil War'),
#          ('http://www.boxofficemojo.com/movies/?id=deadpool2016.htm','Deadpool'),
#          ('http://boxofficemojo.com/movies/?id=biglebowski.htm','The Big Lebowski')]

# # parse_movie_page(movie_dict, url, movie_name):
# test_dict = defaultdict(list)
# for url,name in movies:
#     test_dict = parse_movie_page(test_dict,url,name)

In [11]:
import re
response = requests.get('http://www.boxofficemojo.com/movies/?id=marvel2016.htm')
# should probably check to make sure the response code is good to continue

#'''get the page and soup reference'''#
movie_page = response.text
movie_soup = BeautifulSoup(movie_page)
dtg_string = movie_soup.find(text=re.compile('Genre:')) # finds the label


In [14]:
print(dtg_string.find_parent('td').find('b').get_text(strip=True))

Action / Adventure


In [57]:
response = requests.get('http://www.boxofficemojo.com/movies/?id=pixar2015.htm')
# should probably check to make sure the response code is good to continue

#'''get the page and soup reference'''#
movie_page = response.text
movie_soup = BeautifulSoup(movie_page)
dtg_string = movie_soup.find(text=re.compile('Domestic Total')) # finds the label


In [64]:
dtg_string.find_parent("td").find('b').get_text(strip=True)

u'$431,869,779'

In [59]:
dtg = dtg_string.findNextSibling().text # grabs the next value, which is the dtg


AttributeError: 'NoneType' object has no attribute 'text'

In [69]:
###########################
'''IMPORTANT LINE~~~~~~~~~~~~~'''
###########################

budget_string = soup_star_wars.find(text=re.compile('Production Budget:')).findNextSibling().text


In [70]:
budget_string

u'$245 million'

In [48]:
budget = budget_string.replace('$','').replace(',','')

In [51]:
int(budget.split()[0])*1000000

245000000

In [20]:
###########################
'''IMPORTANT LINE~~~~~~~~~~~~~'''
###########################

# this gets the total lifetime grosses box
print(soup_star_wars.find(class_="mp_box_content"))

<div class="mp_box_content">
<table border="0" cellpadding="0" cellspacing="0">
<tr>
<td width="40%"><b>Domestic:</b></td>
<td align="right" width="35%"> <b>$936,662,225</b></td>
<td align="right" width="25%">   <b>45.3%</b></td>
</tr>
<tr>
<td width="40%">+ <a href="/movies/?page=intl&amp;id=starwars7.htm">Foreign:</a></td>
<td align="right" width="35%"> $1,131,516,000</td>
<td align="right" width="25%">   54.7%</td>
</tr>
<tr>
<td colspan="3" width="100%"><hr/></td>
</tr>
<tr>
<td width="40%">= <b>Worldwide:</b></td>
<td align="right" width="35%"> <b>$2,068,178,225</b></td>
<td width="25%"> </td>
</tr>
</table>
</div>


In [55]:
###########################
'''IMPORTANT LINE~~~~~~~~~~~~~'''
###########################

# this gets the domestic summary
# print(soup_star_wars.find_all(class_='mp_box_content')[1])

'IMPORTANT LINE~~~~~~~~~~~~~'

In [36]:
soup_star_wars.find(text="Opening Weekend:").find_parent("td").find_next_sibling("td").get_text(strip=True)

AttributeError: 'NoneType' object has no attribute 'find_parent'

In [40]:
print(soup_star_wars.find_all(class_='mp_box_content')[1].find('tr').find_all('td')[1].get_text(strip=True))

$247,966,675


In [51]:

print(soup_star_wars)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="en">
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-type">
<title>Star Wars: The Force Awakens (2015) - Box Office Mojo</title>
<style type="text/css">
table.chart-wide { width: 100%; }
</style>
<meta content="star wars: the force awakens, movie, film, box office, result, records, charts, revenue, opening weekend, gross, worldwide, overseas, foreign, news, reviews, articles, stories, story, analysis, revenue, release date, mpaa rating, genre, running time, length, budget, production budget, distributor, studio, buena vista, theatrical summary, theatrical, daily box office results, weekend box office results, weekly box office, weekly box office, international box office summary, worldwide box office summary, similar movies, image gallery, images, pictures, photos, box office mojo" name="keywords">
<meta content="Star Wars: The Force Awakens summary of 

In [42]:
# for element in soup_lebowski.find_all(class_='mp_box_content'):
#     print(element, '\n')

In [12]:
url = 'http://www.boxofficemojo.com/movies/?id=starwars7.htm'

response = requests.get(url)

In [13]:
page_star_wars = response.text
soup_star_wars = BeautifulSoup(page_star_wars)

In [14]:
# for element in soup_star_wars.find_all(class_='mp_box_content'):
#     print(element,'\n')

In [15]:
import re
foreign_total_regex = re.compile('Foreign Total')
soup_star_wars.find(text=foreign_total_regex)


In [16]:
soup_star_wars.find(text="Foreign:").find_parent("td").find_next_sibling("td").get_text(strip=True)


u'$1,131,516,000'

In [17]:
print(soup_star_wars.find(text="Weekend:"))

None


In [44]:
# for a page that doesnt have a foreign total
# soup_lebowski.find(text="Foreign:").find_parent("td").find_next_sibling("td").get_text(strip=True)


In [23]:
soup_star_wars.find(text="Actors:").find_parent("tr").find_all(text=True)[1:]

[]

In [18]:
ftg_string = soup_star_wars.find(text=re.compile('Foreign:'))
# print(ftg_string)

In [19]:
print(ftg_string.findNextSibling())

None
